In [ ]:
!pip install user_agent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime
from functools import cache
from user_agent import generate_user_agent

In [ ]:
def crawler(url):
    time.sleep(2)
    # 製作對網頁請求的假資料，如：假裝是IOS手機、Android手機、mac瀏覽器、desktop
    user_agent = generate_user_agent(device_type='desktop')
    # 代理 ip
    # proxies = {
    #   'http': '123.205.24.240:80',
    # }
    # headers
    headers={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': 'macOS',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': user_agent,
    }
    #對指定的url連結做請求，並給予假資料
    r = requests.get(url, headers=headers, allow_redirects=False)
    r.encoding='utf-8'
    #將網頁作解析，以利找到所需資料
    soup = BeautifulSoup(r.text, 'html.parser')
    #回傳整個網頁的解析
    return soup

def exclude_dot(full):
  name_list = full.rsplit(' ', 1)
  first_middle = name_list[0].split()
  for x in first_middle:
    if x.count('.') > 0 or len(x) == 1:
      first_middle.remove(x)
    if x.count('(') > 0:
      strip_name = x.split('(')[1].strip(')')
      first_middle.remove(x)
      first_middle.append(strip_name)

  if len(first_middle) == 2:
    return ' '.join(first_middle)
  else:
    return first_middle[0]

In [ ]:
author_url, full_name, first_name, last_name = [], [], [], []
for a in range(100):
  url = f'https://www.nber.org/papers/w{31675-a}'
  soup = crawler(url)
  span = soup.find_all('span', class_ = 'page-header__author-item')
  for b in span:
    for_author_url = b.find('a').attrs['href']
    name_count = author_url.count(f'{for_author_url}')
    if name_count == 0:
      author_url.append(for_author_url)
      for_full_name = b.find('a').text
      full_name.append(for_full_name)
      first_name.append(exclude_dot(for_full_name))
      last_name.append(for_full_name.rsplit(' ', 1)[1])

In [ ]:
# Use a single session for performance
sess = requests.Session()

@cache
def get_gender(name: str) -> str:
    resp = sess.get(f"https://api.genderize.io/", params={"name": name})
    resp.raise_for_status()
    return resp.json()["gender"]

def get_gender_or_none(name: str) -> str | None:
    try:
        return get_gender(name)
    except Exception as exc:
        print(f"Failed to get gender for {name}: {exc}")
        return None

In [ ]:
female = []
for c in first_name:
  gender = get_gender_or_none(c)
  if gender is None:
    try:
      gender = get_gender_or_none(c.split()[0])
    except:
      if gender is None:
        try:
          gender = get_gender_or_none(c.split()[1])
        except:
          if gender is None:
            try:
              gender = get_gender_or_none('-'.join(c.split()))
            except:
              if gender is None:
                try:
                  gender = get_gender_or_none(''.join(c.split()))
                except:
                  if gender is None:
                    gender = 'Unknown'

  if gender == 'female':
    female.append(1)
  elif gender == 'male':
    female.append(0)
  else:
    female.append(None)

for d in range(len(female)):
  if female[d] == None:
    print(d, full_name[d], ': search this name')

65 Ruodu Wang : search this name
126 Amiyatosh Purnanandam : search this name


In [ ]:
#手動分類
female.pop(65)
female.insert(65, 0)
female.pop(126)
female.insert(126, 0)

In [ ]:
df = pd.DataFrame({
    'author_url': author_url,
    'full_name': full_name,
    'first_name': first_name,
    'last_name': last_name,
    'female': female})
df.to_csv('nber-author-info.csv', index = False)

In [ ]:
first_name

['Anjali',
 'Benjamin',
 'Fatemeh',
 'Andrew',
 'David',
 'Søren',
 'Ben',
 'Kevin',
 'Michael',
 'Keith',
 'Philip',
 'Michael',
 'Andreas',
 'Kasey',
 'Adrian',
 'Joseph',
 'Haley',
 'Erik',
 'Avinash',
 'Asad',
 'Daley',
 'Haritz',
 'Daniel',
 'Nils',
 'Jae Joon',
 'Meltem',
 'Michael',
 'David',
 'Annamaria',
 'Vansh',
 'Jorge',
 'Maria',
 'Casey',
 'Kevin',
 'Andrés',
 'Luisa',
 'Gueorgui',
 'Richard',
 'Gregory',
 'Celine Yue',
 'David',
 'Jean-Paul',
 'Augustin',
 'Joseph',
 'Nathan',
 'Jonathan',
 'Renee',
 'Lawrence',
 'Peter',
 'Laura',
 'Davin',
 'David',
 'Alberto',
 'Francesco',
 'Ken',
 'Marshall',
 'Mustafa',
 'Noah',
 'Solomon',
 'Daron',
 'Philippe',
 'Lint',
 'David',
 'Job',
 'Aleh',
 'Ruodu',
 'Zhenyuan',
 'Belinda',
 'Francis',
 'Michael',
 'Pierre',
 'Danielle',
 'Eric',
 'Jamie',
 'Anil',
 'May',
 'Oeindrila',
 'Sandy Jo',
 'Anuj',
 'Vincent',
 'Sam',
 'Miyuki',
 'David',
 'Henrik',
 'Camille',
 'Gabriel',
 'Charles',
 'Aaron',
 'Brent',
 'John',
 'Ian',
 'Gregor

In [ ]:
append_exclude_dot(for_full_name)

'Anjali'

In [ ]:
full_name

['Anjali Adukia',
 'Benjamin Feigenberg',
 'Fatemeh Momeni',
 'Andrew Caplin',
 'David J. Deming',
 'Søren Leth-Petersen',
 'Ben Weidmann',
 'Kevin Callison',
 'Michael E. Darden',
 'Keith F. Teltser',
 'Philip Schnorpfeil',
 'Michael Weber',
 'Andreas Hackethal',
 'Kasey Buckles',
 'Adrian Haws',
 'Joseph Price',
 'Haley E.B. Wilbert',
 'Erik Brynjolfsson',
 'Avinash Collis',
 'Asad Liaqat',
 'Daley Kutzman',
 'Haritz Garro',
 'Daniel Deisenroth',
 'Nils Wernerfelt',
 'Jae Joon Lee',
 'N. Meltem Daysal',
 'Michael F. Lovenheim',
 'David N. Wasser',
 'Annamaria Conti',
 'Vansh Gupta',
 'Jorge Guzman',
 'Maria P. Roche',
 'Casey B. Mulligan',
 'Kevin Lang',
 'Andrés Erosa',
 'Luisa Fuster',
 'Gueorgui Kambourov',
 'Richard Rogerson',
 'Gregory W. Brown',
 'Celine Yue Fei',
 'David T. Robinson',
 'Jean-Paul Carvalho',
 'Augustin Bergeron',
 'Joseph Henrich',
 'Nathan Nunn',
 'Jonathan L. Weigel',
 'Renee Bowen',
 'J. Lawrence Broz',
 'B. Peter Rosendorff',
 'Laura Alfaro',
 'Davin Chor',